In [196]:
import os , glob
import nibabel as nib
#os.getcwd()
# os.chdir(r'C:\Users\Armelle\feta1.2.1\derivatives')
# os.getcwd()

In [197]:
import os
import cv2
import glob
import PIL
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from skimage import data
from skimage.util import montage 
import skimage.transform as skTrans
from skimage.transform import rotate
from skimage.transform import resize
from PIL import Image, ImageOps  


# neural imaging
import nilearn as nl
import nibabel as nib
import nilearn.plotting as nlplt
!pip install git+https://github.com/miykael/gif_your_nifti # nifti to gif 
import gif_your_nifti.core as gif2nif


# ml libs
import keras
import keras.backend as K
from keras.callbacks import CSVLogger
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers.experimental import preprocessing


# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)


In [198]:
# DEFINE seg-areas  
SEGMENT_CLASSES = {
    0 : 'CSF',
    1 : 'BS', # or NON-ENHANCING tumor CORE
    2 : 'CBM',
    3 : 'GM' ,# original 4 -> converted into 3 later
    4 : 'SGM',
    5 : 'LV',
    6 : 'WM',
#     7 : 'CBM',
}

# there are 155 slices per volume
# to start at 5 and use 145 slices means we will skip the first 5 and last 5 
VOLUME_SLICES = 256 
VOLUME_START_AT = 0 # first slice of volume that we will include

In [199]:
TRAIN_DATASET_PATH = '../input/fetabraindataset/FeTA_1.2.1_BIDS/feta1.2.1/derivatives/sub-feta001/anat/'

test_image_irm=nib.load(TRAIN_DATASET_PATH + 'sub-feta001_T2w-SR.nii').get_fdata()
test_image_bs=nib.load(TRAIN_DATASET_PATH + 'sub-feta001_T2w-SR_label-BS_mask.nii').get_fdata()
test_image_csf=nib.load(TRAIN_DATASET_PATH + 'sub-feta001_T2w-SR_label-CSF_mask.nii').get_fdata()
# test_image_t1ce=nib.load(TRAIN_DATASET_PATH + 'BraTS20_Training_001/BraTS20_Training_001_t1ce.nii').get_fdata()
# test_image_t2=nib.load(TRAIN_DATASET_PATH + 'BraTS20_Training_001/BraTS20_Training_001_t2.nii').get_fdata()
test_mask=nib.load(TRAIN_DATASET_PATH + 'sub-feta001_T2w-SR_label-CSF_mask.nii').get_fdata()


fig, (ax1,ax2,ax3,ax5) = plt.subplots(1,4, figsize = (20, 10))
slice_w = 25
ax1.imshow(test_image_irm[:,:,test_image_irm.shape[1]//2-slice_w], cmap = 'gray')
ax1.set_title('Image IRM')
ax2.imshow(test_image_bs[:,:,128], cmap = 'gray')
ax2.set_title('mask bs')
ax3.imshow(test_image_csf[:,:,test_image_csf.shape[0]//2-slice_w], cmap = 'gray')
ax3.set_title('mask csf')
# ax4.imshow(test_image_t2[:,:,test_image_t2.shape[0]//2-slice_w], cmap = 'gray')
# ax4.set_title('Image t2')
# ax5.imshow(test_mask[:,:,test_mask.shape[0]//2-slice_w])
# ax5.set_title('Mask')

In [200]:
# Skip 50:-50 slices since there is not much to see
fig, ax1 = plt.subplots(1, 1, figsize = (15,15))
ax1.imshow(rotate(montage(test_image_bs[50:-50,:,:]), 90, resize=True), cmap ='gray')

In [201]:
# Skip 50:-50 slices since there is not much to see
fig, ax1 = plt.subplots(1, 1, figsize = (15,15))
ax1.imshow(rotate(montage(test_mask[60:-60,:,:]), 90, resize=True), cmap ='gray')

In [202]:
# shutil.copy2(TRAIN_DATASET_PATH + 'sub-feta001_T2w-SR.nii', './test_gif_BraTS20_Training_001_flair.nii')
# gif2nif.write_gif_normal('../input/fetabraindataset/FeTA_1.2.1_BIDS/feta1.2.1/derivatives/sub-feta001/anat/sub-feta001_T2w-SR.nii')

![](http://)

In [203]:
niimg = nl.image.load_img(TRAIN_DATASET_PATH + 'sub-feta001_T2w-SR.nii')
nimask = nl.image.load_img(TRAIN_DATASET_PATH + 'sub-feta001_T2w-SR_label-BS_mask.nii')
nimask2 = nl.image.load_img(TRAIN_DATASET_PATH + 'sub-feta001_T2w-SR_label-CSF_mask.nii')

fig, axes = plt.subplots(nrows=5, figsize=(30, 40))


nlplt.plot_anat(niimg,
                title='sub-feta001_T2w-SR.niii plot_anat',
                axes=axes[0])

nlplt.plot_epi(niimg,
               title='sub-feta001_T2w-SR.nii plot_epi',
               axes=axes[1])

nlplt.plot_img(niimg,
               title='sub-feta001_T2w-SR.nii plot_img',
               axes=axes[2])

nlplt.plot_roi(nimask, 
               title='sub-feta001_T2w-SR.nii with mask plot_roi',
               bg_img=niimg, 
               axes=axes[3], cmap='Paired')
nlplt.plot_roi(nimask2, 
               title='sub-feta001_T2w-SR.nii with mask plot_roi',
               bg_img=niimg, 
               axes=axes[4], cmap='Paired')

plt.show()

In [204]:
# dice loss as defined above for 4 classes
def dice_coef(y_true, y_pred, smooth=1.0):
    class_num = 4
    for i in range(class_num):
        y_true_f = K.flatten(y_true[:,:,:,i])
        y_pred_f = K.flatten(y_pred[:,:,:,i])
        intersection = K.sum(y_true_f * y_pred_f)
        loss = ((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))
   #     K.print_tensor(loss, message='loss value for class {} : '.format(SEGMENT_CLASSES[i]))
        if i == 0:
            total_loss = loss
        else:
            total_loss = total_loss + loss
    total_loss = total_loss / class_num
#    K.print_tensor(total_loss, message=' total dice coef: ')
    return total_loss






# Computing Precision 
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    
# Computing Sensitivity      
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())


# Computing Specificity
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [205]:
IMG_SIZE=128

In [206]:
# IMG_SIZE=256

In [223]:
# source https://naomi-fridman.medium.com/multi-class-image-segmentation-a5cc671e647a

def build_unet(inputs, ker_init, dropout):
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(inputs)
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv1)
    
    pool = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(pool)
    conv = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv)
    
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv2)
    
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv3)
    
    
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(pool4)
    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv5)
    drop5 = Dropout(dropout)(conv5)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(UpSampling2D(size = (2,2))(drop5))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv9)
    
    up = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(UpSampling2D(size = (2,2))(conv9))
    merge = concatenate([conv1,up], axis = 3)
    conv = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(merge)
    conv = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv)
    
    conv10 = Conv2D(4, (1,1), activation = 'softmax')(conv)
    
    return Model(inputs = inputs, outputs = conv10)

input_layer = Input((IMG_SIZE, IMG_SIZE, 2))

model = build_unet(input_layer, 'he_normal', 0.2)
# model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics = ['accuracy'] )
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001), metrics = ['accuracy', dice_coef, precision, sensitivity, specificity] )

In [208]:
# plot_model(model, 
#            show_shapes = True,
#            show_dtype=False,
#            show_layer_names = True, 
#            rankdir = 'TB', 
#            expand_nested = False, 
#            dpi = 70)

In [209]:
TRAIN_DATASET_PATH = '../input/fetabraindataset/FeTA_1.2.1_BIDS/feta1.2.1/derivatives'

# lists of directories with studies
train_and_val_directories = [f.path for f in os.scandir(TRAIN_DATASET_PATH) if f.is_dir()]
# print train_and_val_directories[0]


# file BraTS20_Training_355 has ill formatted name for for seg.nii file
# train_and_val_directories.remove(TRAIN_DATASET_PATH+'BraTS20_Training_355')


def pathListIntoIds(dirList):
    x = []
    
    print(len(dirList))
    
    for i in range(0,len(dirList)-10):
#         print(dirList[i][dirList[i].rfind('/')+1:])
        x.append(dirList[i][dirList[i].rfind('/')+1:])
    return x

train_and_test_ids = pathListIntoIds(train_and_val_directories); 
# print(train_and_test_ids)

    
train_test_ids, val_ids = train_test_split(train_and_test_ids,test_size=0.2) 
# print(train_test_ids)
print(val_ids)
if 'sub-feta040'in val_ids:
    val_ids.remove('sub-feta040')
if 'sub-feta041'in val_ids: 
    val_ids.remove('sub-feta041')
if 'sub-feta042'in val_ids:
    val_ids.remove('sub-feta042')
if 'sub-feta043'in val_ids:
    val_ids.remove('sub-feta043')
if 'sub-feta044'in val_ids:
    val_ids.remove('sub-feta044')
if 'sub-feta045'in val_ids:
    val_ids.remove('sub-feta045')
if 'sub-feta046'in val_ids:
    val_ids.remove('sub-feta046')
if 'sub-feta047'in val_ids:
    val_ids.remove('sub-feta047')
if 'sub-feta048'in val_ids:
    val_ids.remove('sub-feta048')
if 'sub-feta049'in val_ids:
    val_ids.remove('sub-feta049')
if 'sub-feta050'in val_ids:
    val_ids.remove('sub-feta050')
print(val_ids)

    
    

    
    
   
    
train_ids, test_ids = train_test_split(train_test_ids,test_size=0.15) 
print(test_ids)
if 'sub-feta040'in test_ids:
    test_ids.remove('sub-feta040')
if 'sub-feta041'in test_ids: 
    test_ids.remove('sub-feta041')
if 'sub-feta042'in test_ids:
    test_ids.remove('sub-feta042')
if 'sub-feta043'in test_ids:
    test_ids.remove('sub-feta043')
if 'sub-feta044'in test_ids:
    test_ids.remove('sub-feta044')
if 'sub-feta045'in test_ids:
    test_ids.remove('sub-feta045')
if 'sub-feta046'in test_ids:
    test_ids.remove('sub-feta046')
if 'sub-feta047'in test_ids:
    test_ids.remove('sub-feta047')
if 'sub-feta048'in test_ids:
    test_ids.remove('sub-feta048')
if 'sub-feta049'in test_ids:
    test_ids.remove('sub-feta049')
if 'sub-feta050'in test_ids:
    test_ids.remove('sub-feta050')
print(val_ids)

if 'sub-feta040'in train_ids:
    train_ids.remove('sub-feta040')
if 'sub-feta041'in train_ids: 
    train_ids.remove('sub-feta041')
if 'sub-feta042'in train_ids:
    train_ids.remove('sub-feta042')
if 'sub-feta043'in train_ids:
    train_ids.remove('sub-feta043')
if 'sub-feta044'in train_ids:
    train_ids.remove('sub-feta044')
if 'sub-feta045'in train_ids:
    train_ids.remove('sub-feta045')
if 'sub-feta046'in train_ids:
    train_ids.remove('sub-feta046')
if 'sub-feta047'in train_ids:
    train_ids.remove('sub-feta047')
if 'sub-feta048'in train_ids:
    train_ids.remove('sub-feta048')
if 'sub-feta049'in train_ids:
    train_ids.remove('sub-feta049')
if 'sub-feta050'in train_ids:
    train_ids.remove('sub-feta050')
# if 'sub-feta016'in train_ids:
#     train_ids.remove('sub-feta016')
print(train_ids)
    
# print(train_ids)
# print(test_ids)

In [210]:
class DataGenerator(keras.utils.all_utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, dim=(IMG_SIZE,IMG_SIZE), batch_size = 1, n_channels = 2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        Batch_ids = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(Batch_ids)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, Batch_ids):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size*VOLUME_SLICES, *self.dim, self.n_channels))
        y = np.zeros((self.batch_size*VOLUME_SLICES, 256, 256))
        Y = np.zeros((self.batch_size*VOLUME_SLICES, *self.dim, 4))

        
        # Generate data
        for c, i in enumerate(Batch_ids):
            case_path = os.path.join(TRAIN_DATASET_PATH, i)

            data_path = os.path.join(f'{case_path}/anat', f'{i}_T2w-SR.nii');
            flair = nib.load(data_path).get_fdata()    

#             data_path = os.path.join(case_path, f'{i}_t1ce.nii');
#             ce = nib.load(data_path).get_fdata()
            
            data_path = os.path.join(f'{case_path}/anat', f'{i}_T2w-SR_label-BS_mask.nii');
            seg = nib.load(data_path).get_fdata()
        
            for j in range(VOLUME_SLICES):
                 X[j +VOLUME_SLICES*c,:,:,0] = cv2.resize(flair[:,:,j+VOLUME_START_AT], (IMG_SIZE, IMG_SIZE));
#                  X[j +VOLUME_SLICES*c,:,:,1] = cv2.resize(ce[:,:,j+VOLUME_START_AT], (IMG_SIZE, IMG_SIZE));

                 y[j +VOLUME_SLICES*c] = seg[:,:,j+VOLUME_START_AT];
                    
        # Generate masks
        y[y==4] = 3;
        mask = tf.one_hot(y, 4);
        Y = tf.image.resize(mask, (IMG_SIZE, IMG_SIZE));
        return X/np.max(X), Y
        
training_generator = DataGenerator(train_ids)
valid_generator = DataGenerator(val_ids)
test_generator = DataGenerator(test_ids)

In [211]:
# show number of data for each dir 
def showDataLayout():
    plt.bar(["Train","Valid","Test"],
    [len(train_ids), len(val_ids), len(test_ids)], align='center',color=[ 'green','red', 'blue'])
    plt.legend()

    plt.ylabel('Number of images')
    plt.title('Data distribution')

    plt.show()
    
showDataLayout()

In [220]:
csv_logger = CSVLogger('training.log', separator=',', append=False)


callbacks = [
#     keras.callbacks.EarlyStopping(monitor='loss', min_delta=0,
#                               patience=2, verbose=1, mode='auto'),
      keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.0001, verbose=1),
#  keras.callbacks.ModelCheckpoint(filepath = 'model_.{epoch:02d}-{val_loss:.6f}.m5',
#                             verbose=1, save_best_only=True, save_weights_only = True)
        csv_logger
    ]

In [213]:
# model.summary()


In [214]:
model.summary()

In [224]:
K.clear_session()

history =  model.fit(training_generator,
                    epochs=50,
                    steps_per_epoch=len(train_ids)//2,
                    callbacks= callbacks,
                    validation_data = valid_generator
                    )  
history=model.fit(training_generator,steps_per_epoch=len(train_ids),
                 callbacks=callbacks,verbose=1)
# model.save("model_x1_1.h5")

In [229]:


acc=hist['accuracy']
val_acc=hist['val_accuracy']

epoch=range(len(acc))

loss=hist['loss']
val_loss=hist['val_loss']

train_dice=hist['dice_coef']
val_dice=hist['val_dice_coef']

f,ax=plt.subplots(1,3,figsize=(16,8))

ax[0].plot(epoch,acc,'b',label='Training Accuracy')
ax[0].plot(epoch,val_acc,'r',label='Validation Accuracy')
ax[0].legend()

ax[1].plot(epoch,loss,'b',label='Training Loss')
ax[1].plot(epoch,val_loss,'r',label='Validation Loss')
ax[1].legend()

ax[2].plot(epoch,train_dice,'b',label='Training dice coef')
ax[2].plot(epoch,val_dice,'r',label='Validation dice coef')
ax[2].legend()

# ax[3].plot(epoch,hist['mean_io_u'],'b',label='Training mean IOU')
# ax[3].plot(epoch,hist['val_mean_io_u'],'r',label='Validation mean IOU')
# ax[3].legend()

# plt.show()

In [226]:
def predictByPath(case_path,case):
#     print(case_path)
    files = next(os.walk(case_path))[2]
    X = np.empty((VOLUME_SLICES, IMG_SIZE, IMG_SIZE, 2))
#     y = np.empty((VOLUME_SLICES, IMG_SIZE, IMG_SIZE))
    
    vol_path = os.path.join(case_path, f'sub-feta{case}_T2w-SR.nii');
#     print(case)
    irm=nib.load(vol_path).get_fdata()
    
#     vol_path = os.path.join(case_path, f'sub-feta{case}_T2w-SR.nii');
#     ce=nib.load(vol_path).get_fdata() 
    
    vol_path = os.path.join(case_path, f'sub-feta{case}_T2w-SR_label-CSF_mask.nii');
    mask_csf=nib.load(vol_path).get_fdata() 
    vol_path = os.path.join(case_path, f'sub-feta{case}_T2w-SR_label-BS_mask.nii');
    mask_bs=nib.load(vol_path).get_fdata() 

    
    for j in range(VOLUME_SLICES):
        X[j,:,:,0] = cv2.resize(irm[:,:,j+VOLUME_START_AT], (IMG_SIZE,IMG_SIZE))
        X[j,:,:,1] = cv2.resize(mask_csf[:,:,j+VOLUME_START_AT], (IMG_SIZE,IMG_SIZE))
#         y[j,:,:] = cv2.resize(mask_bs[:,:,j+VOLUME_START_AT], (IMG_SIZE,IMG_SIZE))
        
  #  model.evaluate(x=X,y=y[:,:,:,0], callbacks= callbacks)
    return model.predict(X/np.max(X), verbose=1)


def showPredictsById(case, start_slice =100 ):
    path = f"../input/fetabraindataset/FeTA_1.2.1_BIDS/feta1.2.1/derivatives/sub-feta{case}/anat/"
#     print(path)
    csf = nib.load(os.path.join(path, f'sub-feta{case}_T2w-SR_label-CSF_mask.nii')).get_fdata()
#     bs = nib.load(os.path.join(path, f'sub-feta{case}_T2w-SR_label-BS_mask.nii')).get_fdata()
#     gm = nib.load(os.path.join(path, f'sub-feta{case}_T2w-SR_label-GM_mask.nii')).get_fdata()
    origImage = nib.load(os.path.join(path, f'sub-feta{case}_T2w-SR.nii')).get_fdata()
    print()
    p = predictByPath(path,case)
#     print(p)

    csf = p[:,:,:,1]
    bs  = p[:,:,:,2]

    plt.figure(figsize=(50, 50))
    f, axarr = plt.subplots(1,3, figsize = (50, 50)) 

    for i in range(3): # for each image, add brain background
        axarr[i].imshow(cv2.resize(origImage[:,:,start_slice+VOLUME_START_AT], (IMG_SIZE, IMG_SIZE)), cmap="gray", interpolation='none')
    
    axarr[0].imshow(cv2.resize(origImage[:,:,start_slice+VOLUME_START_AT], (IMG_SIZE, IMG_SIZE)), cmap="gray")
    axarr[0].title.set_text('Original image')
    curr_csf=cv2.resize(csf[:,:,start_slice+VOLUME_START_AT], (IMG_SIZE, IMG_SIZE), interpolation = cv2.INTER_NEAREST)
    axarr[1].imshow(curr_csf, cmap="Reds", interpolation='none', alpha=0.3) # ,alpha=0.3,cmap='Reds'
    axarr[1].title.set_text('Ground truth')
    axarr[2].imshow(csf[start_slice,:,:], cmap="Reds", interpolation='none', alpha=0.3)
    axarr[2].title.set_text(f'{SEGMENT_CLASSES[0]} predicted')
    plt.show()
    
    
showPredictsById(case=test_ids[0][-3:])
showPredictsById(case=test_ids[1][-3:])
showPredictsById(case=test_ids[2][-3:])



In [228]:
case = test_ids[2][-3:]
path = f"../input/fetabraindataset/FeTA_1.2.1_BIDS/feta1.2.1/derivatives/sub-feta{case}/anat/"
gt = nib.load(os.path.join(path, f'sub-feta{case}_T2w-SR_label-CSF_mask.nii')).get_fdata()
p = predictByPath(path,case)


csf = p[:,:,:,1]



i=100 # slice at
eval_class = 0 #     0 : 'csf',  1 : 'bs',    2 : 'gm',    3 : 'wm'



gt[gt != eval_class] = 1 # use only one class for per class evaluation 

resized_gt = cv2.resize(gt[:,:,i], (IMG_SIZE, IMG_SIZE))

plt.figure()
f, axarr = plt.subplots(1,2) 
axarr[0].imshow(resized_gt, cmap="gray")
axarr[0].title.set_text('ground truth')
axarr[1].imshow(p[i,:,:,eval_class], cmap="gray")
axarr[1].title.set_text(f'predicted class: {SEGMENT_CLASSES[eval_class]}')
plt.show()

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics = ['accuracy', dice_coef, precision, sensitivity, specificity] )
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_generator, batch_size=100, callbacks= callbacks)
print("test loss, test acc:", results)